# step 1: Define Project Goal & Acquire Data

# Step 2: Data Loading and Initial Exploration (EDA)

In [1]:
# importing libraries
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import contractions
from unidecode import unidecode
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from autocorrect import Speller

In [ ]:
# Ensure NLTK data is downloaded (run these once)
# import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

In [2]:
data = pd.read_csv('Train.csv')
data.head(2)

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0


In [3]:
data

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [4]:
data.shape

(40000, 2)

In [5]:
data['text'][0]

'I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.'

# Step 3: Text Preprocessing

In [ ]:
word="I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. No, nor, not liked"


In [6]:
# remove spaces
def remove_spaces(data):
    clean_text = data.replace('\\n',' ').replace('\t',' ').replace('\\',' ')
    return clean_text

# contraction mapping
def expand_text(data):
    expanded_text = contractions.fix(data)
    return expanded_text

# handling accented characters
def handling_accented(data):
    fixed_text = unidecode(data)
    return fixed_text

# Cleaning
stopword_list = stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('nor')
stopword_list.remove('not')

def clean_data(data):
    tokens = word_tokenize(data)
    clean_text = [word.lower() for word in tokens if (word not in punctuation) and (word.lower() not in stopword_list) and (len(word)>2) and (word.isalpha())]
    return clean_text

# autocorrection

def autocorrection(data):
    spell = Speller(lang='en')
    corrected_text = spell(data)
    return corrected_text

# lemmatization

def lemmatization(data):
    lemmatizer = WordNetLemmatizer()
    final_data = []
    for word in data:
        lemmatized_word = lemmatizer.lemmatize(word)
        final_data.append(lemmatized_word)
    return ' '.join(final_data)

In [ ]:
count vectorizer 
tfidf           
>>>> 'w1 w2 w3 w4'

In [7]:
# data leakage
x_train, x_test, y_train, y_test = train_test_split(data.text, data.label,test_size=0.25, random_state=42)

In [8]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to C:\Users\VISHNU
[nltk_data]     KALE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:
clean_text_train = x_train.apply(remove_spaces) #removing spaces from training data
clean_text_test = x_test.apply(remove_spaces) # removing spaces from testing data

clean_text_train = clean_text_train.apply(expand_text)
clean_text_test = clean_text_test.apply(expand_text)

clean_text_train = clean_text_train.apply(handling_accented)
clean_text_test = clean_text_test.apply(handling_accented)

clean_text_train = clean_text_train.apply(clean_data)
clean_text_test = clean_text_test.apply(clean_data)

clean_text_train = clean_text_train.apply(lemmatization)
clean_text_test = clean_text_test.apply(lemmatization)

In [10]:
clean_text_train

26898    fifth grade language art teacher read book stu...
27635    low budget brit pop melodrama focus girl want ...
3036     well watched movie little year ago pulled dust...
5604     would almost give however confusing part well ...
36111    full length feature film world bridge found fi...
                               ...                        
6265     movie one worst movie ever seen life waste tim...
11284    movie inspiring anyone tough jam whether finan...
38158    east side story documentary musical comedy sta...
860      one boot one point doctor assistant refers wor...
15795    movie horrible lighting terrible camera moveme...
Name: text, Length: 30000, dtype: object

In [ ]:
count vectorizer, Tfidf
Parameters

max_df >> by default >> 1.0 >> 100% #This means that terms (words) that appear in more than 95% of the documents will be ignored.
100 documents >> 0.95 >>95%

min_df >> by default >> 1 >> single document # ignores words that are present in only or less documents
100 documents >> 1 >> >>15 >>

max_features >> 1000 #This means that only the top 1000 most frequent terms will be considered.

stopword : 'english'
    
lower_case : True

# Step 4: Feature Engineering / Text Vectorization

In [11]:
# count vectorizer
count = CountVectorizer(max_df = 0.95, max_features=1000)
count_val_train = count.fit_transform(clean_text_train)
count_val_test = count.transform(clean_text_test)


In [12]:
count_val_train

<30000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 1617635 stored elements in Compressed Sparse Row format>

In [13]:
count.get_feature_names_out()

array(['ability', 'able', 'absolutely', 'accent', 'across', 'act',
       'acted', 'acting', 'action', 'actor', 'actress', 'actual',
       'actually', 'adaptation', 'add', 'admit', 'adult', 'adventure',
       'age', 'ago', 'agree', 'air', 'alien', 'alive', 'almost', 'alone',
       'along', 'already', 'also', 'although', 'always', 'amazing',
       'america', 'american', 'among', 'amount', 'animal', 'animation',
       'annoying', 'another', 'answer', 'anyone', 'anything', 'anyway',
       'apart', 'apparently', 'appeal', 'appear', 'appearance', 'appears',
       'appreciate', 'army', 'around', 'art', 'aside', 'ask', 'aspect',
       'atmosphere', 'attack', 'attempt', 'attention', 'audience',
       'average', 'avoid', 'award', 'away', 'awesome', 'awful', 'baby',
       'back', 'background', 'bad', 'badly', 'band', 'barely', 'based',
       'basic', 'basically', 'battle', 'beautiful', 'beauty', 'became',
       'become', 'becomes', 'begin', 'beginning', 'behind', 'belief',
       'be

In [14]:
pd.DataFrame(count_val_train.A,columns = count.get_feature_names_out())

,ability,able,absolutely,accent,across,act,acted,acting,action,actor,...,wrong,wrote,yeah,year,yes,yet,york,young,younger,zombie
0,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
29996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
29998,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


# Step 5: Model Selection

# Step 6: Model Training

# Step 7: Model Evaluation

In [15]:
# build model
count_mnb = MultinomialNB()
count_mnb.fit(count_val_train.A,y_train)
predict_count = count_mnb.predict(count_val_test.A)
accuracy_count = accuracy_score(y_test,predict_count)*100
accuracy_count

83.06

In [16]:
tfidf = TfidfVectorizer(max_df = 0.95, max_features=1000)
tfidf_train = tfidf.fit_transform(clean_text_train)
tfidf_test = tfidf.transform(clean_text_test)

In [17]:
tfidf_train

<30000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1617635 stored elements in Compressed Sparse Row format>

In [18]:
tfidf_train.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
tfidf_train.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
tfidf.get_feature_names_out()

array(['ability', 'able', 'absolutely', 'accent', 'across', 'act',
       'acted', 'acting', 'action', 'actor', 'actress', 'actual',
       'actually', 'adaptation', 'add', 'admit', 'adult', 'adventure',
       'age', 'ago', 'agree', 'air', 'alien', 'alive', 'almost', 'alone',
       'along', 'already', 'also', 'although', 'always', 'amazing',
       'america', 'american', 'among', 'amount', 'animal', 'animation',
       'annoying', 'another', 'answer', 'anyone', 'anything', 'anyway',
       'apart', 'apparently', 'appeal', 'appear', 'appearance', 'appears',
       'appreciate', 'army', 'around', 'art', 'aside', 'ask', 'aspect',
       'atmosphere', 'attack', 'attempt', 'attention', 'audience',
       'average', 'avoid', 'award', 'away', 'awesome', 'awful', 'baby',
       'back', 'background', 'bad', 'badly', 'band', 'barely', 'based',
       'basic', 'basically', 'battle', 'beautiful', 'beauty', 'became',
       'become', 'becomes', 'begin', 'beginning', 'behind', 'belief',
       'be

In [21]:
pd.DataFrame(tfidf_train.A, columns = tfidf.get_feature_names_out())

,ability,able,absolutely,accent,across,act,acted,acting,action,actor,...,wrong,wrote,yeah,year,yes,yet,york,young,younger,zombie
0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.182468,0.0,0.0,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.00000,0.000000,0.0,0.055835,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.137926,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.321531,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.0,0.0,0.000000,0.000000,0.0,0.135608,0.0,0.088057,0.0,0.0,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
29996,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
29997,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.00000,0.000000,0.0,0.074135,0.0,0.0,0.0,0.0,0.0,0.0
29998,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.050570,0.0,0.0,...,0.07508,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# build model
tfidf_mnb = MultinomialNB()
tfidf_mnb.fit(tfidf_train.A,y_train)
predict_tfidf = tfidf_mnb.predict(tfidf_test.A)
accuracy_tfidf = accuracy_score(y_test, predict_tfidf)*100
accuracy_tfidf

83.71

In [ ]:
strin="we are learning NLP"

In [23]:
# ngrams
from nltk.util import ngrams
# [w1,w2,w3]
def splitting_dataframe(data):
    tokens = data.split()
    return tokens

data = clean_text_test.apply(splitting_dataframe)
def ngram_list(data,ngram_range):
    ngram = ngrams(data,ngram_range) # zip file
    ngram_list1 = []
    for ngram1 in ngram: # opening the zip file
        ngram_list1.append(' '.join(ngram1))
    return ngram_list1

unigrams = data.apply(lambda x : ngram_list(x,1))


In [ ]:
ngram_range >> 1 >> unigrams
            >> 2 >> bigrams
            >> 3 >> trigrams

In [24]:
unigrams

32823    [central, theme, movie, seems, confusion, rela...
16298    [excellent, example, cowboy, noir, called, une...
28505    [ending, made, heart, jump, throat, proceeded,...
6689     [chosen, one, appreciate, quality, story, char...
26893    [really, funny, film, especially, second, thir...
                               ...                        
29415    [film, came, gift, offering, blue, unlike, rev...
11359    [first, started, watching, movie, looking, kin...
575      [big, mark, music, neil, young, glowing, prais...
17398    [watching, lady, ermine, wondering, betty, gra...
4189     [crappy, miserably, acted, movie, based, subli...
Name: text, Length: 10000, dtype: object

In [25]:
bigrams = data.apply(lambda x : ngram_list(x,2))
bigrams

32823    [central theme, theme movie, movie seems, seem...
16298    [excellent example, example cowboy, cowboy noi...
28505    [ending made, made heart, heart jump, jump thr...
6689     [chosen one, one appreciate, appreciate qualit...
26893    [really funny, funny film, film especially, es...
                               ...                        
29415    [film came, came gift, gift offering, offering...
11359    [first started, started watching, watching mov...
575      [big mark, mark music, music neil, neil young,...
17398    [watching lady, lady ermine, ermine wondering,...
4189     [crappy miserably, miserably acted, acted movi...
Name: text, Length: 10000, dtype: object

In [26]:
trigrams = data.apply(lambda x : ngram_list(x,3))
trigrams

32823    [central theme movie, theme movie seems, movie...
16298    [excellent example cowboy, example cowboy noir...
28505    [ending made heart, made heart jump, heart jum...
6689     [chosen one appreciate, one appreciate quality...
26893    [really funny film, funny film especially, fil...
                               ...                        
29415    [film came gift, came gift offering, gift offe...
11359    [first started watching, started watching movi...
575      [big mark music, mark music neil, music neil y...
17398    [watching lady ermine, lady ermine wondering, ...
4189     [crappy miserably acted, miserably acted movie...
Name: text, Length: 10000, dtype: object

In [27]:
trigrams = data.apply(lambda x : ngram_list(x,7)) #7-grams
trigrams

32823    [central theme movie seems confusion relations...
16298    [excellent example cowboy noir called unemploy...
28505    [ending made heart jump throat proceeded leave...
6689     [chosen one appreciate quality story character...
26893    [really funny film especially second third fou...
                               ...                        
29415    [film came gift offering blue unlike reviewer,...
11359    [first started watching movie looking kind sub...
575      [big mark music neil young glowing praise, mar...
17398    [watching lady ermine wondering betty grable p...
4189     [crappy miserably acted movie based sublimated...
Name: text, Length: 10000, dtype: object